In [8]:
import torch

def clip_loss(logits, labels, margin=0.4):
    logits_max, _ = logits.max(dim=1, keepdim=True)
    logits = logits - logits_max
    targets = (labels > 0).float()
    losses = -targets * logits + torch.log(1 + torch.exp(logits))
    losses = losses.sum(dim=1)
    return losses.mean()

logits = torch.tensor([[0.1, 0.2, 0.3], [0.4, 0.5, 0.6]])
labels = torch.tensor([1, 2])

In [9]:
logits_max, ex = logits.max(dim=1, keepdim=True)
logits_max

tensor([[0.3000],
        [0.6000]])

In [10]:
logits_norm = logits - logits_max
logits_norm

tensor([[-0.2000, -0.1000,  0.0000],
        [-0.2000, -0.1000,  0.0000]])

In [11]:
targets = (labels > 0).float()
targets

tensor([1., 1.])

In [12]:
labels > 0

tensor([True, True])

In [14]:
torch.exp(logits_norm) # 0 < x < 1

tensor([[0.8187, 0.9048, 1.0000],
        [0.8187, 0.9048, 1.0000]])

In [19]:
1 + torch.exp(logits_norm) # 1 < x+1 < 2

tensor([[1.8187, 1.9048, 2.0000],
        [1.8187, 1.9048, 2.0000]])

In [21]:
torch.log(1 + torch.exp(logits_norm)) # 밑은 e # 0 < y < 1

tensor([[0.5981, 0.6444, 0.6931],
        [0.5981, 0.6444, 0.6931]])

In [26]:
logits = torch.tensor([[0.1, 0.2, 0.3], [0.4, 0.5, 0.6]])
labels = torch.tensor([1, 2])
labels_onehot = torch.nn.functional.one_hot(labels, num_classes=3)
loss = clip_loss(logits, labels_onehot)
loss, labels_onehot

(tensor(1.9857),
 tensor([[0, 1, 0],
         [0, 0, 1]]))

In [29]:
targets = (labels_onehot > 0).float()
targets

tensor([[0., 1., 0.],
        [0., 0., 1.]])

In [30]:
-logits_norm * targets

tensor([[0.0000, 0.1000, -0.0000],
        [0.0000, 0.0000, -0.0000]])

In [31]:
import torch

def cpc_loss(emb1, emb2):
    emb1 = emb1.view(-1, emb1.size(1), emb1.size(2))
    emb2 = emb2.view(-1, emb2.size(1), emb2.size(2))
    emb = torch.cat([emb1, emb2], dim=-1)
    emb = emb.permute(1, 0, 2)
    emb = emb.unsqueeze(1)
    emb_shift = torch.cat([emb[1:], emb[:1]], dim=1)
    logits = torch.matmul(emb, emb_shift.transpose(2, 1)) / (emb.size(-1) ** 0.5)
    logits = logits.view(-1, emb1.size(1))
    labels = torch.arange(emb1.size(1), device=emb1.device).expand(emb1.size(1), emb1.size(1))
    labels = labels == labels.transpose(0, 1)
    labels = labels.view(-1)
    loss = torch.nn.functional.binary_cross_entropy_with_logits(logits, labels.float())
    return loss.mean()

emb1 = torch.randn(512, 256)
emb2 = torch.randn(512, 256)
loss = cpc_loss(emb1, emb2)
print(loss)

IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)

In [37]:
import numpy as np

mat1 = np.array([[1,1,1],[2,2,2]])
mat2 = np.array([[3,3,3],[4,4,4]])

# calculate Euclidean distance between each pair of rows
distances = np.linalg.norm(mat1[:, np.newaxis] - mat2, axis=-1)
mat1[:, np.newaxis].shape

(2, 1, 3)

In [38]:
mat1[:, np.newaxis] - mat2

array([[[-2, -2, -2],
        [-3, -3, -3]],

       [[-1, -1, -1],
        [-2, -2, -2]]])

In [1]:
import torch
from torch.linalg import norm
from torch.nn import CosineEmbeddingLoss
import torch.nn.functional as F

emb1 = torch.rand(32, 32)
emb2 = torch.rand(32, 32)
 
emb1_norm = F.normalize(emb1, dim=-1)
emb2_norm = F.normalize(emb2, dim=-1)
cos_sim_value_1 = torch.matmul(emb1_norm, emb2_norm.T)

dot_product_value_2 = torch.matmul(emb1, emb2.T)
emb1_norm = norm(emb1, dim=-1)
emb2_norm = norm(emb2, dim=-1)
cos_sim_value_2 = dot_product_value_2 / emb1_norm.unsqueeze(1) / emb2_norm.unsqueeze(0)

In [4]:
cos_sim_value_1 == cos_sim_value_2

tensor([[False, False, False,  ..., False, False,  True],
        [ True,  True,  True,  ..., False, False,  True],
        [False,  True, False,  ..., False, False, False],
        ...,
        [ True,  True,  True,  ..., False, False, False],
        [ True, False,  True,  ..., False,  True, False],
        [False, False, False,  ..., False,  True, False]])

In [2]:
cos_sim_value_1

tensor([[0.8399, 0.7862, 0.7911,  ..., 0.7901, 0.7094, 0.8134],
        [0.8672, 0.7421, 0.7859,  ..., 0.7981, 0.7705, 0.8052],
        [0.8195, 0.6941, 0.7518,  ..., 0.7732, 0.7164, 0.7913],
        ...,
        [0.7785, 0.7984, 0.7611,  ..., 0.7822, 0.8325, 0.8305],
        [0.7379, 0.7839, 0.7671,  ..., 0.7255, 0.7672, 0.8097],
        [0.8202, 0.7967, 0.7417,  ..., 0.7708, 0.7560, 0.7876]])

In [3]:
cos_sim_value_2

tensor([[0.8399, 0.7862, 0.7911,  ..., 0.7901, 0.7094, 0.8134],
        [0.8672, 0.7421, 0.7859,  ..., 0.7981, 0.7705, 0.8052],
        [0.8195, 0.6941, 0.7518,  ..., 0.7732, 0.7164, 0.7913],
        ...,
        [0.7785, 0.7984, 0.7611,  ..., 0.7822, 0.8325, 0.8305],
        [0.7379, 0.7839, 0.7671,  ..., 0.7255, 0.7672, 0.8097],
        [0.8202, 0.7967, 0.7417,  ..., 0.7708, 0.7560, 0.7876]])

In [5]:
cos_sim_value_1[0] == cos_sim_value_2[0]

tensor([False, False, False, False, False, False,  True, False, False,  True,
         True, False,  True, False,  True, False, False,  True, False, False,
        False, False, False, False, False, False, False, False,  True, False,
        False,  True])

In [10]:
print(cos_sim_value_1[0][0], cos_sim_value_2[0][0])

tensor(0.8399) tensor(0.8399)


In [15]:
torch.allclose(cos_sim_value_1[0], cos_sim_value_2[0], atol=1e-5)

True

In [16]:
cos_sim_value_1[0].item() == cos_sim_value_2[0].item()


ValueError: only one element tensors can be converted to Python scalars

In [17]:
cos_sim_value_1[0][0], cos_sim_value_2[0][0]

(tensor(0.8399), tensor(0.8399))